In [3]:
%load_ext pep8magic
%matplotlib inline
%pep8
# standard
import csv
import glob
import json
import re
import time
# 3rd-party
from scipy import stats
import pandas as pd
import requests_oauthlib
# self-made
from conf import *

ModuleNotFoundError: No module named 'pep8magic'

In [2]:
# read raw csv
# read the latest file in the csv folder
# file is obtained by Twitter Media Downloader
# https://memo.furyutei.work/entry/20160723/1469282864
dir = sorted(glob.glob('./CSV/*/'))[-1]
file = glob.glob(dir+'*dryrun.csv')[0]
c_d = pd.read_csv(file,
                      parse_dates=["Tweet date"],
                      skiprows=[0,1,2,3])
c_d

NameError: name 'glob' is not defined

In [ ]:
# fetch tweeted user from tweet link URL by API
"""
statuses/show ツイートid1つから取得
900tweet/15m
処理簡単

statuses/lookup ツイートid100つから取得
30000tweet/15m
処理複雑(そうでもない)
"""


twitter = requests_oauthlib.OAuth1Session(CK, CS, t_AT, AS)
def getUser(url, twitter):
    params ={
        'id': re.search(r'^(\d+)', url[::-1]).groups()[0][::-1],
        'trim_user': 'false',
        'include_my_retweet': 'false',
        'include_entities': 'true',
    }
    while True:
        req = twitter.get('https://api.twitter.com/1.1/statuses/show.json',
                      params=params)
        time.sleep(1)
        if req.status_code == 200:
            j = json.loads(req.text)
            try:
                return j['quoted_status']['user']['screen_name']
            except KeyError:
                """
                estimate from the URL in tweet
                when the quoted tweet has been deleted (Reliability: low)
                """
                urls = [u['expanded_url'] for u in j['entities']['urls']]
                cand = [re.search(r'https://twitter.com/([^/]+)/status/\d+',
                                  u) for u in urls]
                cand = [t.groups(1)[0] for t in cand if t]
                if cand:
                    return cand[0]
                else:
                    return 'minyoruminyon'
        else:
            continue

In [ ]:
# make formatted.csv
newcsv = {'Date':c_d['Tweet date'], 
          'URL': c_d['Tweet URL'],
          'User': [],
          'R+L': c_d['Retweets'].fillna(0) + c_d['Likes'].fillna(0),
          'Retweets': c_d['Retweets'],
          'Likes': c_d['Likes']}

# assume user posted a tweet which was RT'd by @minyoruminyon
c = ln = 0
for line in c_d:
    user = re.search(
        r'https://twitter.com/([a-zA-Z0-9_]+)/status/[0-9]+',
        line['Tweet content'])
    if not user:
        user = 'minyoruminyon'
    else:
        user = list(set(user.groups()))
        if user == ['minyoruminyon']:
            user = 'minyoruminyon'
        else:
            user = getUser(line['Tweet URL'])
            time.sleep(1)
            c += 1
            print("%05d:%s        " % (c, user), end="\r")
    print("%05d" % ln, end="\r")    
    newcsv['User'].append(user)
pd.DataFrame(newcsv).fillna(0).to_csv(dir+'formatted.csv', index=None)
pd.read_csv(dir+'formatted.csv')

In [ ]:
# read formatted.csv
f_d = pd.read_csv(dir+'formatted.csv')
# make user.csv
userdata = []
c = 1
tmp = None
for k,v in f_d['User'].value_counts().items():
    userdata.append([c,k,v])
    if tmp != v:
        c += 1
    if not tmp:
        tmp = v
    tmp = v
with open(dir+'user.csv', 'w') as f:
    w = csv.writer(f)
    w.writerow(['Rank', 'User', 'Count'])
    w.writerows(userdata)
pd.read_csv(dir+'user.csv')

In [ ]:
# make buzz.csv
buzz = f_d.sort_values('R+L', ascending=False)
buzz.to_csv(dir+'buzz.csv', index=None)
# reindex
buzz = pd.read_csv(dir+'buzz.csv')
buzz.index = buzz.index + 1
buzz.to_csv(dir+'buzz.csv')
buzz[0:1000].to_csv(dir+'buzz_1000.csv')
pd.read_csv(dir+'buzz_1000.csv')

In [ ]:
# by month
t = pd.read_csv(dir+'formatted.csv')
t['Date'] = pd.to_datetime(t['Date'])
bymonth = []
for y in t['Date'].dt.year.unique()[::-1]:
    for m in range(1,13):
        bymonth.append(["%d/%d" % (y,m),
                       len(t[(t['Date'].dt.year == y)&
                             (t['Date'].dt.month == m)])
                       ])
with open(dir+'month.csv', 'w') as f:
    w = csv.writer(f)
    [w.writerow(i) for i in bymonth]
pd.read_csv(dir+'month.csv')

In [ ]:
# by hour
t = pd.read_csv(dir+'formatted.csv')
t['Date'] = pd.to_datetime(t['Date'])
byhour = []
for h in range(0,24):
    byhour.append(["%02d" % h,
                  len(t[(t['Date'].dt.hour == h)])])
with open(dir+'hour.csv', 'w') as f:
    w = csv.writer(f)
    [w.writerow(i) for i in byhour]
pd.read_csv(dir+'hour.csv')

In [ ]:
# extract some user's one

try:
    os.makedirs(dir+'/user/')
except FileExistsError:
    pass
def extractUser(target):
    n = dir+'/user/%s.csv' % target
    a = pd.read_csv(dir+'formatted.csv')
    a[(a['User'] == target)].to_csv(n, index=False)
    pd.read_csv(n).to_csv(n)
    return pd.read_csv(n)

u = pd.read_csv(dir+'user.csv')
for t in u[(u['Count'] > 9)]['User']:
    print(t,'            ' , end="\r")
    extractUser(t)
print('========END========')

In [ ]:
u = pd.read_csv(dir+'user.csv')
print(len(u),
      len(u[(u['Count']>9)]))

In [ ]:
# make user.md

writer = pytablewriter.MarkdownTableWriter()
writer.from_dataframe(
    u.replace({'User':r'([0-9_a-zA-Z]+)'}, 
              {'User':r'[\1](https://twitter.com/\1)'},regex=True))
writer.dump(dir+"user.md")
writer.write_table()

In [ ]:
# make buzz.md
writer = pytablewriter.MarkdownTableWriter()
b = pd.read_csv(dir+'buzz.csv').replace(
            {'User':r'([0-9_a-zA-Z]+)'}, 
            {'User':r'[\1](https://twitter.com/\1)'},regex=True
    ).replace(
            {'URL':r'(^.*$)'}, 
            {'URL':r'[\1](\1)'},regex=True
    )
writer.from_dataframe(b)
writer.dump(dir+"buzz.md")
writer.from_dataframe(b[0:1000])
writer.dump(dir+"buzz_1000.md")
writer.from_dataframe(b[0:500])
writer.dump(dir+"buzz_500.md")
writer.from_dataframe(b[0:100])
writer.dump(dir+"buzz_100.md")
writer.write_table()

In [ ]:
# make graphs
def showgraph(name, title, xlabel, ylim, dir):
    f=[i.split(',') for i in open(name).readlines()]
    x,y=[],[]
    for i in f:
        x.append(i[0])
        y.append(int(i[1]))

    plt.figure(figsize=(10, 10), dpi=100)
    plt.xlim(0,len(x)-1)
    plt.ylim(0,ylim)
    plt.grid(color='gray')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel('tweet(s)')
    plt.plot(x, y)
    plt.xticks(rotation=90)
    plt.savefig(dir+title+'.png')
showgraph(
    dir+"month.csv",
    """shellgeibot tweet by months
    %s""" % (file[49:][:-11]),
    "YYYY/MM",
    8000,
    dir
)
showgraph(
    dir+"hour.csv",
    """shellgeibot tweet by hours
    %s""" % (file[49:][:-11]),
    "HH",
    4000,
    dir
)